In [1]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [43]:
from IPython.display import HTML

In [46]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
#The raw code for this IPython notebook is by default hidden for easier reading.
#To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [105]:
m_1 = folium.Map(location=[-12.0451206,-77.0981222], tiles='openstreetmap', zoom_start=10)


In [3]:
import geopandas as gpd
#import geoplot.crs as gcrs
#import geoplot as gplt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

comisarias= gpd.read_file('comisarias_fuente_cenepred.shp')

In [4]:
mercados=pd.read_excel('MERCADOS.XLSX')

In [222]:
distritos = gpd.read_file('DISTRITOS.shp')

In [120]:
#distritos = distritos[distritos.DEPARTAMEN.isin(['LIMA','CALLAO'])]

In [ ]:
distritos['centroid_lon'] = distritos['geometry'].centroid.x
distritos['centroid_lat'] = distritos['geometry'].centroid.y
distritos.crs = {'init' :'epsg:4326'}


In [6]:
comisarias = comisarias[comisarias.dpto.isin(['LIMA'])]

In [109]:
comisarias.to_excel('comisarias_lima.xlsx')

In [10]:
comisarias=pd.read_excel('comisarias_lima.xlsx')

In [12]:
comisarias = comisarias[comisarias.dpto.isin(['LIMA'])] 

In [157]:
import geojson

def df_to_geojson(df, properties, lat='centroid_lat', lon='centroid_lon'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['IDDIST']
distritos_json=df_to_geojson(distritos,cols)



In [ ]:
import geopandas as gpd
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('DISTRITOS.shp')
districts_full = districts_full[districts_full.PROVINCIA.isin(['LIMA'])] 
districts = districts_full[["DISTRITO", "geometry"]].set_index("DISTRITO")


In [39]:
# Number of crimes in each police district
plot_dict = comisarias.dist.value_counts()
plot_dict.head()

LIMA                      17
SAN JUAN DE LURIGANCHO    10
COMAS                      7
CHORRILLOS                 7
EL AGUSTINO                7
Name: dist, dtype: int64

In [40]:
import pandas as pd
comisarias=pd.read_excel('comisarias_lima.xlsx')
comisarias = comisarias[comisarias.prov.isin(['LIMA'])] 

In [41]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# Create a base map
m_6 = folium.Map(location=[-12.0451206,-77.0981222], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_6)

# Display the map
m_6

In [24]:
# Create a map
m_1 = folium.Map(location=[-12.0451206,-77.0981222], tiles='cartodbpositron', zoom_start=8)

# Add points to the map
for idx, row in comisarias.iterrows():
    #Marker([row['Lat'], row['Long']]).add_to(m_2)
    Marker([row['Lat'], row['Long']]).add_to(m_1)

# Display the map
m_1.save('MAPA.html')
m_1

#from IPython.display import HTML
#HTML('<iframe src=MAPA.html width=700 height=450></iframe>')


In [29]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

from folium.plugins import MarkerCluster
# Create the map
m_3 = folium.Map(location=[-12.0451206,-77.0981222], tiles='cartodbpositron', zoom_start=13)



# Add points to the map
mc = MarkerCluster()

for idx, row in comisarias.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))

m_3.add_child(mc)

# Display the map
m_3

In [37]:
# Create a base map
m_5 = folium.Map(location=[-12.0451206,-77.0981222], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=comisarias[['Lat', 'Long']], radius=20).add_to(m_5)

# Display the map
m_5

In [45]:
# Create a base map
m_4 = folium.Map(location=[-12.0451206,-77.0981222], tiles='cartodbpositron', zoom_start=13)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_4)

# Add a bubble map to the base map
def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'


for i in range(0,len(comisarias)):
    Circle(
        location=[comisarias.iloc[i]['Lat'], comisarias.iloc[i]['Long']],
        radius=20,
        color=color_producer(comisarias.iloc[i]['objectid'])).add_to(m_4)

# Add points to the map
mc = MarkerCluster()

for idx, row in comisarias.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']], popup=row.nombre))

m_4.add_child(mc)

# casos de covid-19
HeatMap(data=comisarias[['Lat', 'Long']], radius=20).add_to(m_4)



# Display the map
m_4


In [44]:
# Display the map
m_4.save('MAPA.html')
m_4

from IPython.display import HTML
HTML('<iframe src=MAPA.html width=700 height=450></iframe>')

C:\Users\claud\Anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [47]:
from IPython.display import IFrame
IFrame('MAPA.html', width=700, height=350)

In [49]:

html_string = m_4.get_root().render()
html_string

'<!DOCTYPE html>\n<head>    \n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    \n        <script>\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        </script>\n    \n    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.js"></script>\n    <script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>\n    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>\n    <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>\n    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css"/>\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css"/>\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome